# Intro and Notes
If you plan to run this on your own machine, please update the following code block with the appropriate directories, filenames, and options. The rest of the notebook can run seamlessly.

**Please note** that updating all the states took ~19 hours for an i9-7980XE. If you only need to update specific states or already have the state CSVs, be sure to indicate that in the cell below.

County shapefiles for 2020 were retrieved from this link: 
https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_500k.zip

County FEMA data was retrieved from this link:
https://hazards.fema.gov/nri/Content/StaticDocuments/DataDownload//NRI_Table_Counties/NRI_Table_Counties.zip

In [1]:
# Set Directory + Shapefile Name for US County boundaries
shapefile_path = 'counties/cb_2020_us_county_500k.shp'

# Set Directory for Microsoft Building Footprint GeoJSON files
geojson_dir = 'msft_geojson/'

# Define export path for the by-state CSVs with MSFT/OSM areas, as well as the unmapped statistic by county
export_path = 'export_csv/'

# Set path for consolidated CSV with all counties and all areas
consolidated_area_csv = 'all_counties_with_areas.csv'

# Set path for FEMA data
fema_csv = 'fema_data/NRI_Table_Counties.csv'

# Set path for the final CSV with the priority statistic
pri_stat_export_csv = 'priority_statistic_df.csv'

# Define Notebook Run mode and whether you are updating all or some states
method = 'some' #some or all
states = ['Rhode Island', 'District of Columbia', 'Nevada'] #include the states of interest. You may use a blank list if you already have all CSVs.

# Setup
## Imports

In [2]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

import fiona, json, re

from shapely.geometry import shape, Polygon, MultiPolygon, Point
from glob import glob

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# Import county boundaries
master_df = gpd.read_file(shapefile_path)
master_df = master_df[['GEOID', 'NAMELSAD', 'STATE_NAME', 'geometry']]

In [4]:
# Split the 50 States + DC into the official regions/divisions

#Region 1: Northeast
div1 = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island','Vermont']
div2 = ['New Jersey', 'New York', 'Pennsylvania']

#Region 2: Midwest
div3 = ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin']
div4 = ['Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota']

#Region 3: South
div5 = ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'District of Columbia', 'West Virginia']
div6 = ['Alabama', 'Kentucky', 'Mississippi', 'Tennessee']
div7 = ['Arkansas', 'Louisiana', 'Oklahoma', 'Texas']

#Region 4: West
div8 = ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming']
div9 = ['Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']

divs = [div1, div2, div3, div4, div5, div6, div7, div8, div9]

# Function Definitions
## OSM Functions

In [5]:
def get_county_polygons(row, state_df):
    # Use the master shapefile to query OSM by county shape
    mask = state_df[state_df['GEOID']==row['GEOID']].iloc[0]['geometry']
    
    # Return the polygons within the county that aren't points
    try: 
        bld = ox.geometries.geometries_from_polygon(mask,  tags = {'building': True})
        bld = bld[bld['geometry'].apply(lambda x: not isinstance(x, Point))]
        return bld[['geometry']]
    except ox._errors.EmptyOverpassResponse:
        print(str(row['NAMELSAD'])+' was empty')
        return gpd.GeoDataFrame()
    except:
        print("Something else went wrong for "+ str(row['NAMELSAD']))

In [6]:
def get_osm_areas(state):
    # Create a temporary dataframe with the county polygons for the current state
    state_shp_df = master_df[master_df['STATE_NAME'] == state]
    
    # For each county in the state, query the overpass API
    df = gpd.GeoDataFrame()
    for index, row in state_shp_df.iterrows():
        df = pd.concat((df, get_county_polygons(row,state_shp_df)))
    
    # Calculate the area of each of the polygons
    df = df.set_crs(crs=4326, allow_override=True).to_crs(epsg=2163)
    df = df.where(df['geometry'].is_valid)
    df['element_area'] = df['geometry'].area
    df = df.to_crs(crs=4326)
    return df

## MSFT Functions

In [7]:
def get_msft_areas(state):
    # Read the geoJSON file in for the state
    geojson_file = geojson_dir+state.replace(' ','')+'.geojson'
    df = gpd.read_file(geojson_file)
    df = pd.concat({state:df}, names=['STATE_NAME'])
    
    # Calculate the areas of each polygon
    df = df.set_crs(crs=4326, allow_override=True).to_crs(epsg=2163)
    df = df.where(df['geometry'].is_valid)
    df['element_area'] = df['geometry'].area
    df = df.to_crs(crs=4326)
    return df

## General Functions

In [8]:
def get_county_area_sum(row, df):
    try:
        # Create a mask using the county polygon made from the shapefile dataframe
        mask = row['geometry']
        gdf = df.clip(mask)
        
        # Sum up the areas found in that county
        return gdf['element_area'].sum()
    except:
        print('Issue for '+row['NAMELSAD']+ ' in ' + row['STATE_NAME'])
        return 0

In [9]:
def calc_areas(state):
    
    # Create a state dataframe with rows representing counties
    query_df = master_df[master_df['STATE_NAME']==state].copy()
    osm_df = get_osm_areas(state)
    msft_df = get_msft_areas(state)
    
    # Add a column with the Microsoft building area calculation
    query_df['msft_area'] = query_df.apply(lambda row: get_county_area_sum(row, msft_df), axis=1)
    
    # Add a column with the OSM building area calculation
    query_df['osm_area'] = query_df.apply(lambda row: get_county_area_sum(row, osm_df), axis=1)
    
    # Calculate "unmapped" percentage based on area
    query_df['percent_unmapped'] = 100*((query_df['msft_area']-query_df['osm_area'])/(query_df['msft_area']))
    
    # Save as export csv
    query_df.to_csv(export_path+state.replace(' ','')+'.csv')
    return query_df

## Run the Functions per Division
The end state for this section is to have CSVs per state with each row representing a county. Rows will have the Microsoft building area, OSM building area, and unmapped statistic calculated from those two areas. This code block will also generate a consolidated CSV with all counties in the US.

In [10]:
def update_states(method='some', states = ['']):
    area_df = pd.DataFrame()

    if method == 'all':
        for div in divs:
            for state in div:
                area_df = pd.concat([area_df, calc_areas(state)])
                
    else:
        for state in states:
            calc_areas(state)

        def add_csv(main_df, file):
            df = pd.read_csv(file, converters = {'GEOID':str})
            main_df = pd.concat([main_df, df])
            return main_df

        file_list = sorted(glob(export_path + '*.csv'))
        for file in file_list:
            area_df = add_csv(area_df, file)

    area_df.to_csv(consolidated_area_csv)
    return area_df

In [11]:
area_df = update_states(method, states)

C:\Users\catim\anaconda3\lib\site-packages\geopandas\tools\clip.py:67: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[
C:\Users\catim\anaconda3\lib\site-packages\geopandas\tools\clip.py:67: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[
C:\Users\catim\anaconda3\lib\site-packages\geopandas\tools\clip.py:67: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` o

# FEMA Data Import and Merge

In [12]:
fema_df = pd.read_csv(fema_csv, converters = {"STCOFIPS":str}).fillna(0)

In [13]:
## The lists below identifies all the risks to merge in the dataframe.
risks = ["STCOFIPS", "RISK_SCORE", "AVLN_RISKS", "CFLD_RISKS", "CWAV_RISKS", "DRGT_RISKS", "ERQK_RISKS", "HAIL_RISKS", 
         "HWAV_RISKS", "HRCN_RISKS", "ISTM_RISKS", "LNDS_RISKS", "LTNG_RISKS", "RFLD_RISKS", "SWND_RISKS","TRND_RISKS",
         "TSUN_RISKS", "VLCN_RISKS", "WFIR_RISKS", "WNTW_RISKS"]
names = ["avalache_score", "coastal_flooding_score", "cold_wave_score", "drought_score", "earthquake_score", "hail_score", "heat_wave_score", "hurricane_score", 
         "ice_storm_score", "landslide_score", "lightning_score", "riverine_flooding_score", "strong_wind_score", "tornado_score", "tsunami_score", "volcano_score", 
         "wildfire_score", "winter_weather_score"]

fema_df = fema_df[risks] ## Select only relevant risk columns
df = area_df.merge(fema_df, left_on="GEOID", right_on="STCOFIPS", how="left").drop("STCOFIPS", axis=1)

undermapped_ratios = [0, 0.25, 0.5, 0.75, 1.0]

pri_stat_df = pd.DataFrame()
temp_df = df.copy()
for ratio in undermapped_ratios:
    for i in range(len(names)):
        temp_df[names[i]] = temp_df[risks[i+2]]*ratio + temp_df.percent_unmapped*(1-ratio)
    temp_df["ratio"] = ratio*100
    pri_stat_df = pd.concat([pri_stat_df,temp_df])
    
pri_stat_df.to_csv(pri_stat_export_csv)